In [24]:
import os, os.path 
import numpy 
from torch.utils.data import Dataset, DataLoader, random_split
import pickle
from glob import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
import matplotlib
from torch.utils.data import TensorDataset, DataLoader
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

"""Change to the data folder"""
train_path = "./new_train/new_train"
test_path = './new_val_in/new_val_in'

In [8]:
class ArgoverseDataset(Dataset):
    
    """Dataset class for Argoverse"""
    def __init__(self, data_path: str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.data_path = data_path
        self.transform = transform
        self.pkl_list = glob(os.path.join(self.data_path, '*'))
        self.pkl_list.sort()
        
    def __len__(self): #len(val_dataset)
        return len(self.pkl_list)
    
    def __getitem__(self, idx): #val_dataset[0]
        pkl_path = self.pkl_list[idx]
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
        if self.transform:
            data = self.transform(data)
        return data

In [9]:
init_dataset = ArgoverseDataset(data_path=train_path) 
test_dataset  = ArgoverseDataset(data_path=test_path) 
print(len(init_dataset), len(test_dataset))

205942 3200


In [10]:
lengths = [int(len(init_dataset)*0.05) + 1, int(len(init_dataset)*0.95)]
#lengths = [int(len(init_dataset)*0.2) + 1, int(len(init_dataset)*0.5), int(len(init_dataset)*0.3)]

print(lengths)
train_dataset, valid_dataset = random_split(init_dataset, lengths)
#print(len(small_train_dataset), len(test_dataset), len(val_dataset))

[10298, 195644]


# Normalize position

In [30]:
m_distribution = []
p_distribution = []
iterator = tqdm(enumerate(init_dataset), total=int(len(init_dataset)))
for i, val in iterator:
    masked_p_in = val['p_in'][val['car_mask'].reshape(-1) == 1]
    masked_p_in = masked_p_in.reshape(masked_p_in.shape[0] * masked_p_in.shape[1], 2)
    if val['city'] == 'PIT':
        for p in masked_p_in:
            p_distribution.append(p)
    else:
        for p in masked_p_in:
            m_distribution.append(p)

In [32]:
m_distribution = np.array(m_distribution)
p_distribution = np.array(p_distribution)

In [37]:
from sklearn import preprocessing

In [36]:
m_x_mean, m_y_mean = m_distribution[:, 0].mean(), m_distribution[:, 1].mean()

(493.9314568294332, 2627.175757738639)

In [39]:
m_standardizer = preprocessing.StandardScaler().fit(m_distribution)
p_standardizer = preprocessing.StandardScaler().fit(p_distribution)

# Preprocessing data

In [20]:
def data_preprocessing(train_dataset):
    
    arr_feature = []
    arr_label = []

    for i in tqdm(range((len(train_dataset)))):
        
        val = train_dataset[i]

        #filter by car mask
        car_filter = val['car_mask'].reshape(-1) == 1

        p_in = val['p_in'][car_filter]
        v_in = (val['v_in'][car_filter][:, :, 0] ** 2 + val['v_in'][car_filter][:, :, 1] ** 2) ** 0.5
        p_out = val['p_out'][car_filter]

        if val['city'] == 'MIA':
            for c in range(len(p_in)):
                arr_feature.append(np.vstack((p_in[c][:, 0], 
                                    p_in[c][:, 1],
#                                    v_in[c],
#                                    np.zeros(len(p_in[c]))
                                    )).T)
                arr_label.append(p_out[c])
            
        else: #at pit
            city_in = np.zeros(len(p_in))
            for c in range(len(p_in)):
                arr_feature.append(np.vstack((p_in[c][:, 0], 
                                    p_in[c][:, 1],
#                                    v_in[c],
#                                    np.zeros(len(p_in[c])))).T)
                                )).T)
                arr_label.append(p_out[c])
                
    return torch.Tensor(arr_feature), torch.Tensor(arr_label)

In [21]:
inp, out = data_preprocessing(train_dataset)
train_loader = DataLoader(TensorDataset(inp, out),
                        batch_size=256)

In [25]:
class base_model(nn.Module):
    def __init__(self, input_size = 2):
        super().__init__()
        self.hidden_layer_size = 3
        self.hidden_dim = 2048

        self.lstm = nn.LSTM(input_size, self.hidden_dim, num_layers = self.hidden_layer_size, batch_first = True)

        self.linear = nn.Conv1d(self.hidden_dim, 240, 1)


    def forward(self, x):
        x,_ = self.lstm(x)
        x = x.transpose(1, 2)
        x = self.linear(x)
        x = x.transpose(1, 2)
        
        return x
    
    def forward_test(self, x, steps = 30):
        fut = []
        h = torch.zeros((self.hidden_layer_size, len(x), self.hidden_dim)).cuda()
        c = torch.zeros((self.hidden_layer_size, len(x), self.hidden_dim)).cuda()
        for num_iter in range(steps):
            x, (h, c) = self.lstm(x, (h, c))
            x = x[:, -1:]
            x = x.transpose(1, 2)
            x = self.linear(x)
            x = x.transpose(1, 2)
            fut.append(x)
        fut = torch.cat(fut, 1)
        return fut

In [26]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = base_model().cuda()
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
print(model)

cuda
base_model(
  (lstm): LSTM(2, 2048, num_layers=3, batch_first=True)
  (linear): Conv1d(2048, 240, kernel_size=(1,), stride=(1,))
)


In [40]:
model.train()
epochs = 20
loss_ema = -1
loss_ema_test = -1

for i in range(epochs):
    print('epoch', i)
    #iterator = tqdm(train_loader, total=int(len(train_loader)))
    for i_batch, sample_batch in enumerate(train_loader):
        seq, labels = sample_batch
        print(seq.shape), print(labels.shape)
        seq = seq.cuda()
        labels = labels.cuda()
        optimizer.zero_grad()
        full = torch.cat([seq, labels], 1).transpose(1, 2).reshape((-1, 49, 2)).float()
        print(full.shape)
        y_pred = model(full[:, :-1])[:, -30:]
        y_pred = y_pred.reshape((-1, 2, 30)).transpose(1, 2)
        
        print(y_pred.shape, labels.shape)
        loss = loss_function(y_pred, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if loss_ema < 0:
            loss_ema = loss
        loss_ema = loss_ema*0.99 + loss * 0.01
        
        with torch.no_grad():
            y_pred_test = model.forward_test(seq.transpose(1,2).reshape(-1, 19, 240))
            y_pred_test = y_pred_test.reshape((-1, 30, 60, 4)).transpose(1, 2)
            loss_test= loss_function(y_pred_test, labels)
            if loss_ema_test < 0:
                loss_ema_test = loss_test
            loss_ema_test = loss_ema_test*0.99 + loss_test * 0.01
        
        if i_batch % 25 == 0:
            print("Full loss", i, i_batch, loss_ema.item(), loss.item())
            print("Test loss", i, i_batch, loss_ema_test.item(), loss_test.item())

epoch 0
torch.Size([256, 19, 2])
torch.Size([256, 30, 2])
torch.Size([256, 49, 2])
torch.Size([30720, 30, 2]) torch.Size([256, 30, 2])


RuntimeError: The size of tensor a (30720) must match the size of tensor b (256) at non-singleton dimension 0

In [ ]:
torch.save(model.state_dict(), "trial1.pt")